In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from pyoperators import MPI

import qubic
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Calibration.Qselfcal import scatter_plot_FP, get_TEScoordinates_ONAFP
from qubic.lib.Instrument.Qinstrument import QubicInstrument, QubicMultibandInstrument
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing

In [ ]:
nside = 128

# Initialize QUBIC Instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubic.lib.qubicDict()
d.read_from_file(dictfilename)

center = qubic.lib.equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True

d['npointings'] = 20
d['synthbeam_kmax'] = 1
nf_sub = d['nf_sub']
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

In [ ]:
d['nf_sub'] = 20

In [ ]:
# Build MPI communicator and qubic dict
comm = MPI.COMM_WORLD
dict_qubic = d

In [ ]:
# Build scanning strategy
sampling = get_pointing(dict_qubic)

# Build QubicInstrument
q = QubicInstrument(dict_qubic)

In [ ]:
# Number of sub-acquisitions
nsub = d['nf_sub'] / d['nf_recon']

In [ ]:
# Initialize the Qubic multi-frequency acquisition
# I multiply by 2 Nsub to have Nsub bands in each band, the rest of the code will only use the first Nsub bands (meaning the ones in the 150 GHz band)
acq = QubicMultiAcquisitions(dict_qubic, d['nf_sub'], 2, sampling=sampling)

# Build a list that contains all sub-ProjectionOperators, meaning the ProjectionOperators for each sub-bands
Proj_list = []
for i in range(d['nf_sub']):
    op = acq.subacqs[i].get_operator().operands[-1]
    print(op)
    Proj_list.append(op)

# Plot Synthesized Beam

Plot the shape of the beams

In [ ]:
idet = 0

In [ ]:
sb = acq.subacqs[0].instrument[idet].get_synthbeam(acq.subacqs[0].scene)[0]
hp.gnomview(np.log10(sb/np.max(sb)), rot=[0,90], reso=20, min=-3, title="Synthesized Beam - log scale")

In [ ]:
# This is the analytical synthesized for a pixel at the focal plane center 
# without accounting for the primary beam (just the mukltple peaks)
# def sb_noprim(th_deg, nu):
#     th = np.radians(th_deg)
#     lam = 3e8/nu
#     P = 20
#     deltax = 0.013
#     abscissa = np.pi * deltax/lam * th
#     sb = np.sin(P*abscissa)**2 / np.sin(abscissa)**2
#     return sb/np.max(sb)

# N = 5000
# thetas = np.linspace(-30, 30, N)
# phis = 0
# prim_beam = np.zeros((nsub, N))
# thetas_peak = []
# thetas_test = np.linspace(-15, -5, N)

# colors = ['blue', 'red', 'green', 'orange']
# plt.figure(figsize=(10, 8))
# for i in range(nsub):
#     prim_beam[i] = acq.multiinstrument[i].primary_beam(np.radians(np.abs(thetas)), phis)
#     ind_peak = np.where((prim_beam[i]*sb_noprim(thetas_test, acq.allnus[i]*1e9)) == np.max(prim_beam[i]*sb_noprim(thetas_test, acq.allnus[i]*1e9)))[0]
#     thetas_peak.append(thetas_test[ind_peak])
#     plt.plot(thetas, prim_beam[i], '--', color = colors[i], label='Prim. Beam - nu={0:6.1f} GHz'.format(acq.allnus[i]))
#     plt.plot(thetas, prim_beam[i] * sb_noprim(thetas, acq.allnus[i]*1e9), color = colors[i], label='Synth. Beam - nu={0:6.1f} GHz'.format(acq.allnus[i]))
# plt.xlim(-20, 20)
# plt.xlabel('$\\theta$ [deg]')
# plt.ylabel('Normalized beam')
# plt.legend()

# Plot Primary and Synthesized Beams

In [ ]:
idet = 231

In [ ]:
isubband = 0

In [ ]:
# Compute Synthetisized Beam for a given detector 
synth = acq.multiinstrument[isubband].get_synthbeam(acq.scene)[idet]

npix = hp.nside2npix(nside)
prim_beam = np.zeros(npix)

# Compute Primary Beam
prim = acq.multiinstrument[isubband].primary_beam
for i in range(npix):
    theta, phi = hp.pix2ang(nside, i)
    prim_beam[i] = prim(theta, phi)

In [ ]:
# Compute the position of the secondary peaks using _peak_angles_kmax
th, ph = acq.subacqs[isubband].instrument._peak_angles_kmax(acq.subacqs[isubband].instrument.synthbeam.kmax, 
                                                     acq.subacqs[isubband].instrument.horn.spacing, 
                                                     acq.subacqs[isubband].instrument.horn.angle, 
                                                     acq.subacqs[isubband].instrument.filter.nu, 
                                                     acq.subacqs[isubband].instrument.detector.center)
# Compute the amplitude of each peaks
val = np.array(prim(th, ph), dtype=float, copy=False)

In [ ]:
# Compute the position and the amplitude of the secondary peaks using _peak_angles
# This function uses _peak_angles_kmax
thetas, phis, vals = acq.subacqs[isubband].instrument._peak_angles(acq.scene, acq.allnus[isubband]*1e9, 
                                    acq.subacqs[isubband].instrument.detector.center,
                                    acq.subacqs[isubband].instrument.synthbeam,
                                    acq.subacqs[isubband].instrument.horn,
                                    acq.subacqs[isubband].instrument.primary_beam)

In [ ]:
# Compare the result, to see if these functions compute properly the peak position
hp.gnomview(prim_beam, rot=[0,90], reso=20, title="Primary Beam")
hp.projplot(thetas[idet], phis[idet], 'rx', lonlat=False)
hp.gnomview(np.log10(synth/np.max(synth)), rot=[0, 90], min=-3, reso=15, title="Synthesized Beam - peak_angles")
hp.projplot(thetas[idet], phis[idet], 'rx', lonlat=False)
hp.gnomview(np.log10(synth/np.max(synth)), rot=[0, 90], min=-3, reso=15, title="Synthesized Beam - peak_angles_kmax")
hp.projplot(th[idet], ph[idet], 'rx', lonlat=False)

In [ ]:
# Relative diff between primary beams
isubband_a, isubband_b = 3, 2

prim_a = acq.multiinstrument[isubband_a].primary_beam
prim_b  = acq.multiinstrument[isubband_b].primary_beam

npix = hp.nside2npix(nside)
prim_beam_a = np.zeros(npix)
prim_beam_b = np.zeros(npix)

for i in range(npix):
    theta, phi = hp.pix2ang(nside, i)
    prim_beam_a[i] = prim(theta, phi)
    prim_beam_b[i] = prim_b(theta, phi)

hp.gnomview(prim_beam_a, rot=[0,90], reso=20, sub = (1,3,1), title=f"PB at {acq.allnus[isubband_a].round(2)} GHz")
hp.gnomview(prim_beam_b, rot=[0,90], reso=20, sub = (1,3,2), title=f"PB at {acq.allnus[isubband_b].round(2)} GHz")

hp.gnomview((prim_beam_a / prim_beam_b), rot=[0,90], reso=20, sub = (1,3,3), title=f" PBs at {acq.allnus[isubband_a].round(2)} and {acq.allnus[isubband_b].round(2)} GHz Divided")

# Compare Primary and Synthesized Beams evolution with frequency

In [ ]:
n_pointings = Proj_list[0].matrix.data.r11.shape[0] / 992

In [ ]:
 #matrix.data.index su samo indexirane kombinacije detektora*vrijeme
 #u r11 peakovi su spremljeni tako da ide (0 - 18) za prvi detektor i 18 pointinga
#znaci za 

#for det in range(992):
 #   for peak in range(sh[1]):
  #      for sample in range(det*18, (det+1)*18): #za 18 pointinga i nulti detektor ovo ce bit od 0 do 17
   #         mapPitPi[P0.matrix.data.index[sample,peak]

In [ ]:
i = 0 # frequency choice, lowest subband

In [ ]:
idet = 0

#### Peak_angles_kmax returns thetas and phis of the 9 retained peaks per detector (unsorted):

In [ ]:
thetas_pakm, phis_pakm = acq.subacqs[0].instrument._peak_angles_kmax(acq.subacqs[i].instrument.synthbeam.kmax, 
                                                           acq.subacqs[i].instrument.horn.spacing,
                                                           acq.subacqs[i].instrument.horn.angle, 
                                                          acq.allnus[i]*1e9,
                                                          acq.subacqs[i].instrument.detector.center)

In [ ]:
thetas_pakm[idet, :]

#### Primary beam gives intesity of peaks at given theta and phi. Returned thetas, phis, and vals are not sorted:

In [ ]:
vals_pm = np.array(acq.multiinstrument[0].primary_beam(thetas_pakm, phis_pakm), dtype=float, copy=False)

In [ ]:
vals_pm[idet, :] 

#### Finally, peak_angles sorts thetas, phis, and vals by the intensity of each peak. 

In [ ]:
def _argsort_reverse(a, axis=-1):
    i = list(np.ogrid[[slice(x) for x in a.shape]])
    i[axis] = a.argsort(axis)[:, ::-1]
    return i

In [ ]:
vals = np.zeros((vals_pm.shape))

In [ ]:
vals[~np.isfinite(vals_pm)] = 0
index = _argsort_reverse(vals_pm)
thetas = thetas_pakm[tuple(index)]
phis = phis_pakm[tuple(index)]
vals = vals_pm[tuple(index)]
cumval = np.cumsum(vals, axis=-1)
imaxs = np.argmax(cumval >= acq.subacqs[0].instrument.synthbeam.fraction * cumval[:, -1, None],
                  axis=-1) + 1
imax = max(imaxs)

# slice initial arrays to discard the non-significant peaks
thetas = thetas[:, :imax]
phis = phis[:, :imax]
vals = vals[:, :imax]

In [ ]:
thetas_pakm[idet], vals_pm[idet]

In [ ]:
thetas[idet], vals[idet]

In [ ]:
th_pa, ph_pa, v_pa = acq.subacqs[i].instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        acq.subacqs[i].instrument.detector.center,
                                        acq.subacqs[i].instrument.synthbeam,
                                        acq.subacqs[i].instrument.horn,
                                        acq.subacqs[i].instrument.primary_beam)

In [ ]:
thetas[idet] == th_pa[idet]

#### Thetas and phis remain the same, but peak_angles additionaly scales each val. This adds a $(\frac{1}{\nu})^2$ dependancy.

In [ ]:
vals[idet], v_pa[idet]

In [ ]:
acq.subacqs[i].instrument.synthbeam.peak150.solid_angle * (150 / (acq.allnus[i])) ** 2 / acq.scene.solid_angle * len(acq.subacqs[i].instrument.horn)

In [ ]:
solid_angle = acq.subacqs[i].instrument.synthbeam.peak150.solid_angle * (150 / (acq.allnus[i])) ** 2 # znaci za duplo vecu frekvenciju promjena je 1/4
vals_scaled = vals * solid_angle / acq.scene.solid_angle * len(acq.subacqs[i].instrument.horn)

In [ ]:
print('vals scaled')
print(vals_scaled[idet])
print('vals from peak_angles')
print(v_pa[idet])
print('ratio vals to v_pa')
print(vals_scaled[idet] / v_pa[idet])

#### Finally, the same scaled values are found in the data of the Projection operator:

In [ ]:
Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, :]

In [ ]:
i = 0

In [ ]:
th, ph, v = acq.subacqs[i].instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        acq.subacqs[i].instrument.detector.center,
                                        acq.subacqs[i].instrument.synthbeam,
                                        acq.subacqs[i].instrument.horn,
                                        acq.subacqs[i].instrument.primary_beam)

#### The primary beam returns the sorted but unscaled values (because we feed it sorted thetas and phis):

In [ ]:
prim_freq[i](th[idet, :], ph[idet, :])

In [ ]:
vals[idet]

In [ ]:
scaling_factor = [acq.subacqs[j].instrument.synthbeam.peak150.solid_angle * (150 / (acq.allnus[j])) ** 2 / acq.scene.solid_angle * len(acq.subacqs[j].instrument.horn) for j in range(len(acq.subacqs))]
np.array(scaling_factor)

In [ ]:
freqs = acq.allnus
freqs

In [ ]:
one_over_nu_squared = 1 / (acq.allnus)**2

In [ ]:
one_over_nu_squared

In [ ]:
scaling_factor / one_over_nu_squared

In [ ]:
#therefore scaling_factor is 29977432.60401282 * one_over_nu_squared
29977432.60401282 * one_over_nu_squared

#### Dividing by scaling factors gives the original unscaled vals, so the dependancy with $(\frac{1}{\nu})^2$ works as expected:

In [ ]:
Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, :] / scaling_factor[i]

In [ ]:
ipeak = 0
time_index = 0
idet = 0
i = 0

In [ ]:
acq.allnus

In [ ]:
[Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak] for i in range(20)]

In [ ]:
[Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak] / scaling_factor[i] for i in range(20)] 

### This means that the data is NOT following a power-law 1/f squared! If we scale an array with 1/f squared and it rises than falls it means the original data has a slower power law 1/f^n where n<2

In [ ]:
acq.subacqs[i].instrument.synthbeam.peak150.solid_angle

In [ ]:
idet = 231
limit = 10

In [ ]:
time_index = 0

for ipeak in range(thetas.shape[1]):
    peak_amplitude = [] 
    peak_amplitude_unscaled = []
    fig, ax1 = plt.subplots()

    prim_freq = [acq.multiinstrument[freq].primary_beam for freq in range(len(acq.multiinstrument[:limit]))]
    v_freq = []
    for i in range(len(acq.multiinstrument[:limit])):
        th, ph, v = acq.subacqs[i].instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        acq.subacqs[i].instrument.detector.center,
                                        acq.subacqs[i].instrument.synthbeam,
                                        acq.subacqs[i].instrument.horn,
                                        acq.subacqs[i].instrument.primary_beam)
        v_freq.append(v[idet, ipeak])
        
        #ax1.plot(acq.allnus[i], prim_freq[i](th[idet, ipeak], ph[idet, ipeak]), '.b', color = 'blue')
  
        peak_amplitude.append(Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak])
        peak_amplitude_unscaled.append(Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak] / scaling_factor[i])
    #ax1.plot(acq.allnus, v_freq, 'x', label = 'Peak Amplitude (Q_instrument._peak_angles)')

    ax1.set_xlabel('Frequency (GHz)')
    ax1.plot(acq.allnus[:limit], peak_amplitude_unscaled, 'x', color = 'green')
    ax1.set_ylabel('Primary beam amplitudes', color = 'green')
    ax1.set_title('Primary beam and peak amplitude at peak %d for detector %d' % (ipeak, idet))
    ax1.legend(loc = 'best')
    
    ax2 = ax1.twinx()
    ax2.plot(acq.allnus[:limit], peak_amplitude, '.r')
    ax2.set_ylabel("Peak Amplitude after solid angle correction", color = 'r')
    
    plt.show()

In [ ]:
time_index = 0

for ipeak in range(1):
    peak_amplitude = [] 
    peak_amplitude_scaled = []
    fig, ax1 = plt.subplots()

    prim_freq = [acq.multiinstrument[freq].primary_beam for freq in range(len(acq.multiinstrument))]
    v_freq = []
    for i in range(len(acq.multiinstrument)):
        th, ph, v = acq.subacqs[i].instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        acq.subacqs[i].instrument.detector.center,
                                        acq.subacqs[i].instrument.synthbeam,
                                        acq.subacqs[i].instrument.horn,
                                        acq.subacqs[i].instrument.primary_beam)
        v_freq.append(v[idet, ipeak])
        
        #ax1.plot(acq.allnus[i], prim_freq[i](th[idet, ipeak], ph[idet, ipeak]), '.b', color = 'blue')
  
        peak_amplitude.append(Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak])
        peak_amplitude_scaled.append(Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak] / scaling_factor[i])
    #ax1.plot(acq.allnus, v_freq, 'x', label = 'Peak Amplitude (Q_instrument._peak_angles)')

    ax1.set_xlabel('Frequency (GHz)')
    #ax1.plot(acq.allnus[:], peak_amplitude_scaled, 'x', color = 'green')
    ax1.plot(acq.allnus[:], one_over_nu_squared, 'x')
    ax1.plot(acq.allnus[:], 1 / acq.allnus[:]**2, alpha = 0.5)
    ax1.set_ylabel('One over nu squared', color = 'blue')
    ax1.set_title('Peak amplitude')
    ax1.legend(loc = 'best')
    
    ax2 = ax1.twinx()
    ax2.plot(acq.allnus[:], peak_amplitude, '.r')
    ax2.set_ylabel("Peak Amplitude (r11)", color = 'r')
    
    plt.show()

In [ ]:
time_index = 10

for ipeak in range(thetas.shape[1]):
    peak_amplitude = [] 
    fig, ax1 = plt.subplots()

    prim_freq = [acq.multiinstrument[i].primary_beam for i in range(len(acq.multiinstrument))]
    v_freq = []
    for i in range(len(acq.multiinstrument)):
        th, ph, v = acq.subacqs[i].instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        acq.subacqs[i].instrument.detector.center,
                                        acq.subacqs[i].instrument.synthbeam,
                                        acq.subacqs[i].instrument.horn,
                                        acq.subacqs[i].instrument.primary_beam)
        v_freq.append(v[idet, ipeak])
  
        peak_amplitude.append(Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak])
    ax1.plot(acq.allnus, v_freq, 'x')

    ax1.set_xlabel('Frequency (GHz)')
    ax1.set_ylabel('Peak Amplitude (Q_instrument._peak_angles)', color = 'blue')
    ax1.set_title('Primary beam and peak amplitude at peak %d for detector %d' % (ipeak, idet))
    
    ax2 = ax1.twinx()
    ax2.plot(acq.allnus[:], peak_amplitude, '.r')
    ax2.set_ylabel("Peak Amplitude (r11)", color = 'r')
    
    plt.show()

In [ ]:
time_index = 0

for ipeak in range(thetas.shape[1]):
    peak_amplitude = [] 
    fig, ax1 = plt.subplots()

    prim_freq = [acq.multiinstrument[i].primary_beam for i in range(len(acq.multiinstrument))]
    v_freq = []
    for i in range(len(acq.multiinstrument)):
        th, ph, v = acq.subacqs[i].instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        acq.subacqs[i].instrument.detector.center,
                                        acq.subacqs[i].instrument.synthbeam,
                                        acq.subacqs[i].instrument.horn,
                                        acq.subacqs[i].instrument.primary_beam)
        v_freq.append(v[idet, ipeak])
        ax1.plot(acq.allnus[i], prim_freq[i](th[idet, ipeak], ph[idet, ipeak]), '.b')
  
        peak_amplitude.append(Proj_list[i].matrix.data.r11[int(idet*(n_pointings))+time_index, ipeak])

    ax1.set_xlabel('Frequency (GHz)')
    ax1.set_ylabel('Primary beam (Q_instrument.primary_beam)', color = 'blue')
    ax1.set_title('Primary beam and peak amplitude at peak %d for detector %d' % (ipeak, idet))
    ax1.legend(loc = 'best')
    
    ax2 = ax1.twinx()
    ax2.plot(acq.allnus, v_freq, 'x', color = 'g', label = 'Peak Amplitude (Q_instrument._peak_angles)')
    ax2.set_ylabel("Peak Amplitude (Q_instrument._peak_angles)", color = 'g')
    
    plt.show()

# Build TOD

In [ ]:
# Create uniform maps of "1"
input_maps = np.ones((d['nf_sub'], hp.nside2npix(nside), 3))
#input_maps[..., 1] *= 0 

# Build the list of sub-TODs
tod = []
for i in range(d['nf_sub']):
    tod.append(Proj_list[i](input_maps[i]))

In [ ]:
hp.gnomview(input_maps[0,:,0], rot = center, reso = 20)

In [ ]:
istokes = 0

In [ ]:
tod[isubband][idet,time_index,0] # svaki detektor vidi jednu vrijednost priko svih pointinga za pojedini stokes (medju stokesima se razlikuje)

# Relative difference between succesive sub-TODs

In [ ]:
tod[i][:, time_index, 0].mean(), tod[i][:, time_index, 0].std()

In [ ]:
tod[i][:, time_index, 1].mean(), tod[i][:, time_index, 1].std()

In [ ]:
plt.figure()
mean_diff = []
for i in range(d['nf_sub']-1):
    diff = np.abs((tod[i+1][:, time_index, istokes] - tod[i][:, time_index, istokes])/tod[i][:, time_index, istokes])
    # Need to remove 'nan' due to division by 0
    # Mean on detectors
    #mean_diff.append(np.mean(diff[~np.isnan(diff)]))
    mean_diff = np.mean(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
#plt.plot(acq.allnus[0:-1], mean_diff, '.')
plt.title(r"$H = \mathcal{P}_{roj}$ - Intensity map")
plt.ylabel(r'$\frac{| H(Imap)^{i+1} - H(Imap)^i |}{| H(Imap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

In [ ]:
plt.figure()
for i in range(d['nf_sub']-1):
    diff = np.abs((tod[i+1][:, time_index, 1] - tod[i][:, time_index, 1]))#/tod[i][:, time_index, 1])
    # Need to remove 'nan' due to division by 0
    # Mean on detectors
    mean_diff = np.median(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
plt.title(r"$H = \mathcal{P}_{roj}$ - Q map")
plt.ylabel(r'$\frac{| H(Qmap)^{i+1} - H(Qmap)^i |}{| H(Qmap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

In [ ]:
plt.figure()
for i in range(d['nf_sub']-1):
    diff = np.abs((tod[i+1][:, time_index, 2] - tod[i][:, time_index, 2]))#/tod[i][:, time_index, 1])
    # Need to remove 'nan' due to division by 0
    # Mean on detectors
    mean_diff = np.mean(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
plt.title(r"$H = \mathcal{P}_{roj}$ - U map")
plt.ylabel(r'$| H(Umap)^{i+1} - H(Umap)^i |$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

In [ ]:
plt.figure()
idet = 245
for idet in range(200, 210):
    for i in range(d['nf_sub']-1):
        diff = (tod[i+1][idet, time_index, 1] - tod[i][idet, time_index, 1])/tod[i][idet, time_index, 1]#np.abs((tod[i+1][idet, 0, 1] - tod[i][idet, 0, 1])/tod[i][idet, 0, 1])
        plt.plot(acq.allnus[i], diff, '.b')
    plt.title(r"$H = \mathcal{P}_{roj}$ - Q map - " + f'Detector index = {idet}')
    plt.ylabel(r'$\frac{ H(Qmap)^{i+1} - H(Qmap)^i |}{| H(Qmap)^{i}}$', fontsize=15, labelpad=10)
    plt.xlabel('Frequency (GHz)')
    plt.show()

In [ ]:
np.abs(tod[i][:, time_index, 1]).min()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
scatter_plot_FP(q, xTES, yTES, tod[i][:, time_index, 0], frame='ONAFP', s=150, title=r'Intensity map - $\frac{ \mathcal{P}_{roj}(Imap)^{n+1} - \mathcal{P}_{roj}(Imap)^{n} }{ \mathcal{P}_{roj}(Imap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
scatter_plot_FP(q, xTES, yTES, tod[11][:, 10, 1], frame='ONAFP', s=150, title=r'Q map - $\frac{ \mathcal{P}_{roj}(Imap)^{n+1} - \mathcal{P}_{roj}(Imap)^{n} }{ \mathcal{P}_{roj}(Imap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
scatter_plot_FP(q, xTES, yTES, tod[i][:, time_index, 2], frame='ONAFP', s=150, title=r'Intensity map - $\frac{ \mathcal{P}_{roj}(Imap)^{n+1} - \mathcal{P}_{roj}(Imap)^{n} }{ \mathcal{P}_{roj}(Imap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

# Relative difference on the focal plane

Plot the difference between two successive tods, at a given time, on the focal plane. The polarisation case seems very strange with respect to the intensity case.

In [ ]:
xTES, yTES, vertex = get_TEScoordinates_ONAFP(q)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
#these are frequencies not detectors
ifreq_a, ifreq_b = 3, 2
relative_diff = (tod[ifreq_a]-tod[ifreq_b])#/tod[ifreq_b]*100
print(relative_diff.shape)
scatter_plot_FP(q, xTES, yTES, np.mean(relative_diff[...,0], axis=1), frame='ONAFP', s=150, title=r'Intensity map - $\frac{ \mathcal{P}_{roj}(Imap)^{n+1} - \mathcal{P}_{roj}(Imap)^{n} }{ \mathcal{P}_{roj}(Imap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
np.argmax((tod[ifreq_a][:, 0, 1]-tod[ifreq_b][:, 0, 1])/tod[ifreq_b][:, 0, 1]*100)

In [ ]:
tod[ifreq_b][472,0,1]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (tod[ifreq_a]-tod[ifreq_b])#/tod[ifreq_a]
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 1], frame='ONAFP', s=150, title=r'Q map - $\frac{| \mathcal{P}_{roj}(Qmap)^{n+1} - \mathcal{P}_{roj}(Qmap)^{n} |}{| \mathcal{P}_{roj}(Qmap)^{n} |} x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (tod[idet_a]-tod[idet_b])/tod[idet_b]*100
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 2], frame='ONAFP', s=150, title=r'U map - $\frac{| \mathcal{P}_{roj}(Umap)^{n+1} - \mathcal{P}_{roj}(Umap)^{n} |}{| \mathcal{P}_{roj}(Umap)^{n} |} x 100$', fig=fig, ax = ax, unit='%')

# Projection angle

In [ ]:
P = Proj_list[-1]

alpha_from_r22 = 0.5 * np.arccos(P.matrix.data.r22 / P.matrix.data.r11)
alpha_from_r32 = 0.5 * np.arcsin(- P.matrix.data.r32 / P.matrix.data.r11)

cos2_sin2 = (P.matrix.data.r22**2 + P.matrix.data.r32**2) / P.matrix.data.r11**2

In [ ]:
cos2_sin2[0]

In [ ]:
plt.plot(cos2_sin2[0], '.', label=r'$cos^2 + sin^2$')
plt.hlines(1, 0, (2*dict_qubic['synthbeam_kmax'] + 1)**2, 'grey', linestyle='--', alpha=0.5)
plt.xlabel('Peaks index')
plt.legend()

# Peak Amplitude evolution with frequency

P.matrix.data is build such that you have all the points for the same detector at all the different time samples, then the next detector at all the different time samples, etc.

In [ ]:
peak_index = 0
N_sample = 0
time_index = 0
det_index = 205
for peak_index in range((2*dict_qubic['synthbeam_kmax']+1)**2):
    plt.figure()
    peak_amplitude = []
    for i in range(nsub):
        peak_amplitude.append(Proj_list[i].matrix.data.r11[(det_index*N_sample)+time_index, peak_index])

    plt.plot(acq.allnus[:nsub], peak_amplitude)
    plt.xlabel("Frequency (GHz)")
    plt.ylabel("Peak Amplitude")
    plt.title(f"Detector Index = {det_index} | Time Sample Index = {time_index} | Peak Index = {peak_index}")
    plt.show()

# TOD change with frequency  

In [ ]:
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubic.lib.qubicDict()
d.read_from_file(dictfilename)

center = qubic.lib.equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True

d['npointings'] = 20
d['synthbeam_kmax'] = 1
nf_sub = d['nf_sub']
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

comm = MPI.COMM_WORLD

### I will still try to remain at the same frequency. This code stores the Projection operator at 148.83 ghz

In [ ]:
Proj_list_nsub = []
nsubs_no = []
j = 0
proj_nus = []
for i in range(2, 50, 4):
    nsubs_no.append(i)
    d['nf_sub'] = i
    sampling = get_pointing(d)
    q = QubicInstrument(d)
    acq = QubicMultiAcquisitions(d, i, 2, sampling=sampling)
    op = acq.subacqs[j].get_operator().operands[-1] #always take the lowest frequency
    Proj_list_nsub.append(op)
    proj_nus.append(acq.allnus[j])
    j = j + 1 # ode ide step iz range / 4
    print(i/2)

In [ ]:
proj_nus

In [ ]:
for i in range(2, 50, 4):
    print(i/2)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(nsubs_no, proj_nus)
axes[0].set_title('Middle frequency by Nsub')
axes[1].scatter(nsubs_no[1:], proj_nus[1:])
axes[1].set_title('Without first one (zoom)')

In [ ]:
input_maps = np.ones((d['nf_sub'], hp.nside2npix(nside), 3))
tod_lowfrq_nsub = []
for i in range(len(Proj_list_nsub)):
    tod_lowfrq_nsub.append(Proj_list_nsub[i](input_maps[i]))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

istokes_lbl = ['I', 'Q', 'U']
time_index = 0
for istokes in range(3):
    mean_diff_per_det = []
    
    for i in range(len(tod_lowfrq_nsub)):
        diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) / np.abs(tod_lowfrq_nsub[-1][:, time_index, istokes])
        mean_diff_per_det = np.mean(diff_per_det)
        axes[istokes].plot(nsubs_no[i], mean_diff_per_det, '.')
    axes[istokes].set_title(r"$\mathcal{P}_{roj}$ - " + istokes_lbl[istokes])
    axes[0].set_ylabel(r'$\frac{| P(Imap)^{i} - P(Imap)^{max} |}{| P(Imap)^{max} |}$', fontsize=15, labelpad=10)
    axes[istokes].set_xlabel('Nsub')
    axes[istokes].set_yscale('log')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

istokes_lbl = ['I', 'Q', 'U']
time_index = 0
for istokes in range(3):
    mean_diff_per_det = []
    
    for i in range(len(tod_lowfrq_nsub)):
        diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) 
        mean_diff_per_det = np.mean(diff_per_det)
        axes[istokes].plot(nsubs_no[i], mean_diff_per_det, '.')
    axes[istokes].set_title(r"$\mathcal{P}_{roj}$ - " + istokes_lbl[istokes])
    axes[0].set_ylabel(r'$| P(Imap)^{i} - P(Imap)^{max} |$', fontsize=15, labelpad=10)
    axes[istokes].set_xlabel('Nsub')
    axes[istokes].set_yscale('log')

In [ ]:
plt.figure()
mean_diff_per_det = []
istokes = 0
time_index = 0
for i in range(len(Proj_list_nsub)):
    diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) / np.abs(tod_lowfrq_nsub[-1][:, time_index, istokes])
    mean_diff_per_det = np.mean(diff_per_det)
    plt.plot(nsubs_no[i], mean_diff_per_det, '.')
#plt.plot(acq.allnus[0:-1], mean_diff, '.')
plt.title(r"$\mathcal{P}_{roj}$ - Intensity")
plt.ylabel(r'$\frac{| P(Imap)^{i} - P(Imap)^{max} |}{| P(Imap)^{max} |}$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')

In [ ]:
plt.figure()
mean_diff_per_det = []
istokes = 1
time_index = 0
for i in range(len(Proj_list_nsub)):
    diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) / np.abs(tod[-1][:, time_index, istokes])
    mean_diff_per_det = np.mean(diff_per_det)
    plt.plot(nsubs_no[i], mean_diff_per_det, '.')
#plt.plot(acq.allnus[0:-1], mean_diff, '.')
plt.title(r"$\mathcal{P}_{roj}$ - Q")
plt.ylabel(r'$\frac{| P(Imap)^{i} - P(Imap)^{max} |}{| P(Imap)^{max} |}$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')

In [ ]:
plt.figure()
mean_diff_per_det = []
istokes = 2
time_index = 0
for i in range(len(Proj_list_nsub)):
    diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) / np.abs(tod[-1][:, time_index, istokes])
    mean_diff_per_det = np.mean(diff_per_det)
    plt.plot(nsubs_no[i], mean_diff_per_det, '.')
#plt.plot(acq.allnus[0:-1], mean_diff, '.')
plt.title(r"$\mathcal{P}_{roj}$ - U")
plt.ylabel(r'$\frac{| P(Imap)^{i} - P(Imap)^{max} |}{| P(Imap)^{max} |}$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')

# Linear spacing

In [ ]:
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubic.lib.qubicDict()
d.read_from_file(dictfilename)

center = qubic.lib.equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True

d['npointings'] = 20
d['synthbeam_kmax'] = 1
nf_sub = d['nf_sub']
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

comm = MPI.COMM_WORLD

### I will still try to remain at the same frequency. This code stores the Projection operator at 148.83 ghz

In [ ]:
Proj_list_nsub = []
nsubs_no = []
j = 0
proj_nus = []
for i in range(2, 100, 16):
    nsubs_no.append(i)
    d['nf_sub'] = i
    sampling = get_pointing(d)
    q = QubicInstrument(d)
    acq = QubicMultiAcquisitions(d, i, 2, sampling=sampling)
    op = acq.subacqs[j].get_operator().operands[-1] #always take the middle frequency
    Proj_list_nsub.append(op)
    proj_nus.append(acq.allnus[j])
    j = j + 4 # ode ide step iz range / 4

In [ ]:
proj_nus

In [ ]:
for i in range(2, 100, 16):
    print(i/2)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].scatter(nsubs_no, proj_nus)
axes[0].set_title('Middle frequency by Nsub')
axes[1].scatter(nsubs_no[1:], proj_nus[1:])
axes[1].set_title('Without first one (zoom)')

In [ ]:
input_maps = np.ones((d['nf_sub'], hp.nside2npix(nside), 3))
tod_lowfrq_nsub = []
for i in range(len(Proj_list_nsub)):
    tod_lowfrq_nsub.append(Proj_list_nsub[i](input_maps[i]))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

istokes_lbl = ['I', 'Q', 'U']
time_index = 0
for istokes in range(3):
    mean_diff_per_det = []
    
    for i in range(len(tod_lowfrq_nsub)):
        diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) / np.abs(tod_lowfrq_nsub[-1][:, time_index, istokes])
        mean_diff_per_det = np.mean(diff_per_det)
        axes[istokes].plot(nsubs_no[i], mean_diff_per_det, '.')
    axes[istokes].set_title(r"$\mathcal{P}_{roj}$ - " + istokes_lbl[istokes])
    axes[0].set_ylabel(r'$\frac{| P(Imap)^{i} - P(Imap)^{max} |}{| P(Imap)^{max} |}$', fontsize=15, labelpad=10)
    axes[istokes].set_xlabel('Nsub')
    #axes[istokes].set_yscale('log')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

istokes_lbl = ['I', 'Q', 'U']
time_index = 0
for istokes in range(3):
    mean_diff_per_det = []
    
    for i in range(len(tod_lowfrq_nsub)):
        diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) 
        mean_diff_per_det = np.mean(diff_per_det)
        axes[istokes].plot(nsubs_no[i], mean_diff_per_det, '.')
    axes[istokes].set_title(r"$\mathcal{P}_{roj}$ - " + istokes_lbl[istokes])
    axes[0].set_ylabel(r'$| P(Imap)^{i} - P(Imap)^{max} |$', fontsize=15, labelpad=10)
    axes[istokes].set_xlabel('Nsub')
    #axes[istokes].set_yscale('log')

In [ ]:
plt.figure()
mean_diff_per_det = []
istokes = 0
time_index = 0
for i in range(len(Proj_list_nsub)):
    diff_per_det = np.abs((tod_lowfrq_nsub[i][:, time_index, istokes] - tod_lowfrq_nsub[-1][:, time_index, istokes])) / np.abs(tod_lowfrq_nsub[-1][:, time_index, istokes])
    mean_diff_per_det = np.mean(diff_per_det)
    plt.plot(nsubs_no[i], mean_diff_per_det, '.')
#plt.plot(acq.allnus[0:-1], mean_diff, '.')
plt.title(r"$\mathcal{P}_{roj}$ - Intensity")
plt.ylabel(r'$\frac{| P(Imap)^{i} - P(Imap)^{max} |}{| P(Imap)^{max} |}$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')

In [ ]:
plt.figure()
mean_diff = []
istokes = 0

for i in range(d['nf_sub']-1):
    diff = np.abs((tod[i+1][:, time_index, istokes] - tod[i][:, time_index, istokes]))#/tod[i][:, time_index, istokes])
    # Need to remove 'nan' due to division by 0
    # Mean on detectors
    #mean_diff.append(np.mean(diff[~np.isnan(diff)]))
    mean_diff = np.mean(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
#plt.plot(acq.allnus[0:-1], mean_diff, '.')
plt.title(r"$H = \mathcal{P}_{roj}$ - Intensity map")
plt.ylabel(r'$| H(Imap)^{i+1} - H(Imap)^i |$', fontsize=15, labelpad=10)
plt.ylim(0,400)
plt.xlabel('Frequency (GHz)')

In [ ]:
plt.figure()
istokes = 1

for i in range(d['nf_sub']-1):
    diff = np.abs((tod[i+1][:, time_index, 1] - tod[i][:, time_index, 1]))#/tod[i][:, time_index, 1])
    # Need to remove 'nan' due to division by 0
    # Mean on detectors
    mean_diff = np.mean(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
plt.title(r"$H = \mathcal{P}_{roj}$ - Q map")
plt.ylabel(r'$| H(Imap)^{i+1} - H(Imap)^i |$', fontsize=15, labelpad=10)
plt.ylim(0,400)
plt.xlabel('Frequency (GHz)')

In [ ]:
plt.figure()
istokes = 2

for i in range(d['nf_sub']-1):
    diff = np.abs((tod[i+1][:, time_index, istokes] - tod[i][:, time_index, istokes]))#/tod[i][:, time_index, 1])
    # Need to remove 'nan' due to division by 0
    # Mean on detectors
    mean_diff = np.mean(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
plt.title(r"$H = \mathcal{P}_{roj}$ - U map")
plt.ylabel(r'$| H(Umap)^{i+1} - H(Umap)^i |$', fontsize=15, labelpad=10)
plt.ylim(0,400)
plt.xlabel('Frequency (GHz)')

In [ ]:
plt.figure()
idet = 245
for idet in range(200, 210):
    for i in range(d['nf_sub']-1):
        diff = (tod[i+1][idet, time_index, 1] - tod[i][idet, time_index, 1])/tod[i][idet, time_index, 1]#np.abs((tod[i+1][idet, 0, 1] - tod[i][idet, 0, 1])/tod[i][idet, 0, 1])
        plt.plot(acq.allnus[i], diff, '.b')
    plt.title(r"$H = \mathcal{P}_{roj}$ - Q map - " + f'Detector index = {idet}')
    plt.ylabel(r'$\frac{ H(Qmap)^{i+1} - H(Qmap)^i |}{| H(Qmap)^{i}}$', fontsize=15, labelpad=10)
    plt.xlabel('Frequency (GHz)')
    plt.show()

In [ ]:
xTES, yTES, vertex = get_TEScoordinates_ONAFP(q)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
#these are frequencies not detectors
ifreq_a, ifreq_b = 3, 2
relative_diff = (tod[ifreq_a]-tod[ifreq_b])#/tod[ifreq_b]*100
print(relative_diff.shape)
scatter_plot_FP(q, xTES, yTES, np.mean(relative_diff[...,0], axis=1), frame='ONAFP', s=150, title=r'Intensity map - $ \mathcal{P}_{roj}(Imap)^{n+1} - \mathcal{P}_{roj}(Imap)^{n} $', fig=fig, ax = ax, unit='%')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 12))
relative_diff = (tod[ifreq_a]-tod[ifreq_b])#/tod[ifreq_a]

scatter_plot_FP(q, xTES, yTES, tod[ifreq_a][:, 0, 1], frame='ONAFP', s=150, title=r'Q map - $| \mathcal{P}_{roj}(Qmap)^{n+1}  | $', fig=fig, ax = axes[0], unit='%')
scatter_plot_FP(q, xTES, yTES, tod[ifreq_b][:, 0, 1], frame='ONAFP', s=150, title=r'Q map - $|  \mathcal{P}_{roj}(Qmap)^{n} | $', fig=fig, ax = axes[1], unit='%')
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 1], frame='ONAFP', s=150, title=r'Q map - $| \mathcal{P}_{roj}(Qmap)^{n+1} - \mathcal{P}_{roj}(Qmap)^{n} | $', fig=fig, ax = axes[2], unit='%')

## Lets see if the error also disappeared from H:

## First, for fixed frequency (similar to how we looked at P)

In [ ]:
H_list_nsub = []
nsubsH_no = []
j = 0
H_nus = []
for i in range(2, 100, 16):
    nsubsH_no.append(i)
    d['nf_sub'] = i
    sampling = get_pointing(d)
    q = QubicInstrument(d)
    acq = QubicMultiAcquisitions(d, i, 2, sampling=sampling)
    op = acq.subacqs[j].get_operator() #always take the middle frequency
    H_list_nsub.append(op)
    j = j + 4 # ode ide step iz range / 4

In [ ]:
tod_H = []
for i in range(len(H_list_nsub)):
    tod_H.append(H_list_nsub[i](input_maps[i]))

In [ ]:
mean_diff = []
plt.figure()
for i in range(len(H_list_nsub)-1):
    diff = np.abs((tod_H[-1][:, time_index] - tod_H[i][:, time_index]))#/tod_H[-1][:, time_index])
    mean_diff.append(np.mean(diff))
    plt.plot(nsubsH_no[i], np.mean(diff), '.')
plt.title(r"H with linear spacing")
plt.ylabel(r'$| H(Imap)^{max} - H(Imap)^i |$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')

## Seems like the problem disappeared but lets try to look at real sky (CMB instead of uniform) to replicate the exact plot Tom made

In [ ]:
from qubic.lib.MapMaking.FrequencyMapMaking.Qspectra_component import CMBModel

np.random.seed(3)
nsubs_no_final = []
TODs_final = []
check_acq_bandwidth = []
cl_cmb = CMBModel(None).give_cl_cmb(r=0, Alens=1)
cmb_map = hp.synfast(cl_cmb, nside, new=True, verbose=False).T

for i in range(2, 33, 4):
    nsubs_no_final.append(i)
    sky_config = {'cmb': 3}
    
    d['nf_sub'] = i
    Qubic_sky = qubic.lib.QskySim.Qubic_sky(sky_config, d)
    sky_map = Qubic_sky.get_simple_sky_map()

    # sky_map = sky_map[None] + np.zeros((i, hp.nside2npix(nside), 3))
    print(sky_map.shape)
    
    sampling = get_pointing(d)
    acq = QubicMultiAcquisitions(d, i, 2, sampling=sampling)
    summ = 0
    for i in range(len(acq.H)):
        summ = summ + (acq.H[i](sky_map[i])) # we have to add the delta nu to summation acq.subacqs[i].get_filter_operator()
    #TOD, _ = acq.get_observation(sky_map, noiseless=d['noiseless'])
    TODs_final.append(summ)
    check_acq_bandwidth.append(acq.subacqs[0].get_filter_operator())

In [ ]:
plt.figure(figsize = (10,10))
for i in range(len(TODs_final)):
    plt.plot(range(len(TODs_final[0][:30,0])), TODs_final[i][:30,0], label = 'nsub '+str(nsubs_no_final[i]))
plt.legend(loc = 'best')
plt.title('First 30 pointings of TOD per nsub')
plt.ylabel('TOD')
plt.xlabel('pointing')

In [ ]:
plt.figure(figsize = (10,10))
plt.plot(range(len(TODs_final[0][0,:])), TODs_final[0][0,:], label = 'nsub '+str(nsubs_no_final[i]))
plt.legend(loc = 'best')
plt.title('First 30 pointings of TOD per nsub')
plt.ylabel('TOD')
plt.xlabel('pointing')

In [ ]:
time_index = 0

In [ ]:
mean_diff = []
for i in range(len(TODs_final)):
    diff = np.abs((TODs_final[i][:, time_index] - TODs_final[-1][:, time_index])/TODs_final[-1][:, time_index])
    mean_diff.append(np.mean(diff))

In [ ]:
plt.figure()
plt.plot(nsubs_no_final, mean_diff, '.')
plt.title(r"H with linear spacing (all frequencies) TODs")
plt.ylabel(r'$\frac{| H(Imap)^{max} - H(Imap)^i |}{H(Imap)^{max}}$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')
plt.yscale('log')

In [ ]:
nsubs_no_qubicsoft = [2, 10, 18, 26, 34, 42, 50, 58, 74, 90, 130]
mean_diff_qubic_soft = [0.23328019, 0.05593016, 0.02726084, 0.01733298, 0.015180879912922126, 0.011851136572479338, 0.009573197553846079, 0.009183327526276779, 0.006966553326405692, 0.00631729577513964, 0.004212596032614286]

In [ ]:
plt.figure()
plt.plot(nsubs_no_final, mean_diff, '.', label = 'New H (from MapMaking or reorg)')
plt.plot(nsubs_no_qubicsoft, mean_diff_qubic_soft, '.', label = 'Old H')
plt.title(r"H with linear spacing (all frequencies)")
plt.ylabel(r'$\frac{| H(Imap)^{max} - H(Imap)^i |}{H(Imap)^{max}}$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')
plt.legend(loc = 'best')
#plt.yscale('log')

In [ ]:
plt.figure()
plt.plot(nsubs_no_final, mean_diff, '.', label = 'New H (from MapMaking or reorg)')
plt.plot(nsubs_no_qubicsoft, mean_diff_qubic_soft, '.', label = 'Old H')
plt.title(r"H with linear spacing (all frequencies)")
plt.ylabel(r'$\frac{| H(Imap)^{max} - H(Imap)^i |}{H(Imap)^{max}}$', fontsize=15, labelpad=10)
plt.xlabel('Nsub')
plt.legend(loc = 'best')
plt.yscale('log')

In [ ]:
d